In [ ]:
import pandas as pd
import numpy as np

In [ ]:
GENR14raw = pd.read_csv("14GENR.csv", header=None)
GENR14raw.head(20)

In [9]:
def data_clean(election):
    #Drop irrelevant columns
    if len(election.columns) == 9: clean = election.drop([2, 3, 6, 8], axis=1)
    else: clean = election.drop([4, 6], axis=1)
    #Create column names
    clean.columns = ['Precinct', 'Race', 'Statistic', 'Party', 'Count']
    #Drop irrelevant rows
    clean = clean[clean.Race != "Race Statistics"]
    #Get rid of white space at beginning or end of values
    for c in ['Precinct', 'Race', 'Statistic', 'Party']:
        clean[c] = pd.core.strings.str_strip(clean[c])
    #Replace "Write-in 70" etc. with "Write-in" - STILL NOT WORKING
    clean.Statistic = clean.Statistic.replace("Write-in.*", "Write-in")
    
    #Pivot table so each precinct is one row with attributes for many elections
    pivot = pd.pivot_table(clean, values='Count', columns=['Race', 'Statistic', 'Party'], index='Precinct')
    
    #Calculate total votes by race
    votes = pivot.drop(["Number of Precincts for Race",
                                 "Number of Precincts Reporting",
                                 "Registered Voters",
                                 "Times Counted"], axis=1, level=1)
    totals = votes.sum(level=0, axis=1)
    
    #Calculate percentages for each candidate
    pct = votes / totals
    
    #Calculate voter turnout
    #use "US REPRESENTATIVE" in general elections
    #use "US REPRESENTATIVE (R)" + "US REPRESENATIATIVE (adl)" in primaries
    if "US REPRESENTATIVE" in pivot.columns.levels[0]:
        rep = "US REPRESENTATIVE"
        turnout = pivot[rep]["Times Counted"]['NP'] / pivot[rep]["Registered Voters"]['NP']
    else:
        turnout = (pivot["US REPRESENTATIVE (R)"]["Times Counted"]['NP'] + pivot["US REPRESENTATIVE (ADL)"]["Times Counted"]['NP'])\
        / pivot["US REPRESENTATIVE (R)"]["Registered Voters"]['NP']
    
    pct['Turnout'] = turnout
    return pct
#Returns only percentages and totals - ideally would include raw vote totals, but having difficulty with join


In [10]:
PRIM14raw = pd.read_csv("14PRIM.csv", header=None)
GENR16raw = pd.read_csv("16GENR.csv", header=None)
PRIM16raw = pd.read_csv("16PRIM.csv", header=None)

In [11]:
elections_dict = {}
for e in ('GENR14', 'PRIM14', 'GENR16', 'PRIM16'):
    el = locals()[e+'raw']
    elections_dict[e] = data_clean(el)

In [12]:
elections_dict['GENR14']

Race                       Ballot Measure 2 - 13PSUM            \
Statistic                                         NO       YES   
Party                                             NP        NP   
Precinct                                                         
01-446 Aurora                               0.452193  0.547807   
01-455 Fairbanks No. 1                      0.380208  0.619792   
01-465 Fairbanks No. 2                      0.339683  0.660317   
01-470 Fairbanks No. 3                      0.424547  0.575453   
01-475 Fairbanks No. 4                      0.387097  0.612903   
01-480 Fairbanks No. 5                      0.491826  0.508174   
01-485 Fairbanks No. 6                      0.386059  0.613941   
01-490 Fairbanks No. 7                      0.383621  0.616379   
01-495 Fairbanks No. 10                     0.473404  0.526596   
02-345 Badger No. 2                         0.409950  0.590050   
02-355 Fairbanks No. 8                      0.451557  0.548443   
02-365 Fairbanks No. 9                      0.489437  0.510563   
02-375 Fort Wainwright                      0.535714  0.464286   
03-130 Badger No. 1                         0.476699  0.523301   
03-135 Chena Lakes                          0.539859  0.460141   
03-165 Newby                                0.516493  0.483507   
03-175 North Pole                           0.546490  0.453510   
03-183 Plack                                0.504193  0.495807   
04-230 Ester                                0.355263  0.644737   
04-240 Farmers Loop                         0.420635  0.579365   
04-250 Goldstream No. 1                     0.289417  0.710583   
04-260 Goldstream No. 2                     0.293463  0.706537   
04-265 Steese East-Gilmore                  0.488424  0.511576   
04-270 Steese West                          0.440578  0.559422   
04-280 University Hills                     0.304147  0.695853   
05-580 Airport                              0.454545  0.545455   
05-582 Chena                                0.381941  0.618059   
05-586 Geist                                0.446532  0.553468   
05-587 Lakeview                             0.380282  0.619718   
05-588 Pike                                 0.385650  0.614350   
...                                              ...       ...   
District 37 - Absentee                      0.417582  0.582418   
District 37 - Early Voting                  0.368421  0.631579   
District 37 - Question                      0.332031  0.667969   
District 38 - Absentee                      0.531250  0.468750   
District 38 - Early Voting                  0.482759  0.517241   
District 38 - Question                      0.489362  0.510638   
District 39 - Absentee                      0.454955  0.545045   
District 39 - Early Voting                  0.368231  0.631769   
District 39 - Question                      0.417431  0.582569   
District 4 - Absentee                       0.359223  0.640777   
District 4 - Early Voting                   0.363128  0.636872   
District 4 - Question                       0.283117  0.716883   
District 40 - Absentee                      0.405882  0.594118   
District 40 - Early Voting                  0.558824  0.441176   
District 40 - Question                      0.316327  0.683673   
District 5 - Absentee                       0.356077  0.643923   
District 5 - Early Voting                   0.374289  0.625711   
District 5 - Question                       0.273810  0.726190   
District 6 - Absentee                       0.483208  0.516792   
District 6 - Early Voting                   0.394144  0.605856   
District 6 - Question                       0.355140  0.644860   
District 7 - Absentee                       0.533502  0.466498   
District 7 - Early Voting                   0.573218  0.426782   
District 7 - Question                       0.407332  0.592668   
District 8 - Absentee                       0.536649  0.463351   
District 8 - Early Voting               

In [13]:
elections_dict['PRIM14']

Race                              GOVERNOR (ADL)                    \
Statistic                  Clift, Carolyn F. \C" Mallott, Byron I.   
Party                                        LIB               DEM   
Precinct                                                             
01-446 Aurora                           0.215596          0.637615   
01-455 Fairbanks No. 1                  0.234043          0.531915   
01-465 Fairbanks No. 2                  0.171429          0.628571   
01-470 Fairbanks No. 3                  0.152866          0.668790   
01-475 Fairbanks No. 4                  0.197802          0.637363   
01-480 Fairbanks No. 5                  0.148649          0.594595   
01-485 Fairbanks No. 6                  0.150000          0.612500   
01-490 Fairbanks No. 7                  0.219178          0.602740   
01-495 Fairbanks No. 10                 0.195652          0.586957   
02-345 Badger No. 2                     0.356098          0.424390   
02-355 Fairbanks No. 8                  0.131250          0.681250   
02-365 Fairbanks No. 9                  0.140541          0.675676   
02-375 Fort Wainwright                  0.416667          0.250000   
03-130 Badger No. 1                     0.335616          0.410959   
03-135 Chena Lakes                      0.365591          0.236559   
03-165 Newby                            0.319328          0.453782   
03-175 North Pole                       0.396226          0.358491   
03-183 Plack                            0.340659          0.340659   
04-230 Ester                            0.151832          0.670157   
04-240 Farmers Loop                     0.074468          0.776596   
04-250 Goldstream No. 1                 0.106250          0.706250   
04-260 Goldstream No. 2                 0.148289          0.673004   
04-265 Steese East-Gilmore              0.161943          0.631579   
04-270 Steese West                      0.165333          0.624000   
04-280 University Hills                 0.133333          0.680000   
05-580 Airport                          0.120000          0.720000   
05-582 Chena                            0.151575          0.643701   
05-586 Geist                            0.149038          0.610577   
05-587 Lakeview                         0.242424          0.454545   
05-588 Pike                             0.187500          0.708333   
...                                          ...               ...   
District 37 - Absentee                  0.087719          0.763158   
District 37 - Early Voting              0.000000          0.857143   
District 37 - Question                  0.175676          0.702703   
District 38 - Absentee                  0.130952          0.797619   
District 38 - Early Voting              0.200000          0.600000   
District 38 - Question                  0.155844          0.779221   
District 39 - Absentee                  0.071429          0.857143   
District 39 - Early Voting              0.097222          0.888889   
District 39 - Question                  0.083333          0.833333   
District 4 - Absentee                   0.146341          0.626016   
District 4 - Early Voting               0.074627          0.817164   
District 4 - Question                   0.229008          0.580153   
District 40 - Absentee                  0.133333          0.752381   
District 40 - Early Voting              0.090909          0.818182   
District 40 - Question                  0.135135          0.684685   
District 5 - Absentee                   0.120000          0.706667   
District 5 - Early Voting               0.053892          0.796407   
District 5 - Question                   0.198020          0.613861   
District 6 - Absentee                   0.150538          0.704301   
District 6 - Early Voting               0.138889          0.787037   
District 6 - Question                   0.216981          0.650943   
District 7 - Absentee                   0.358209          0.462687   
District 7

In [14]:
elections_dict['GENR16']

Race                       Ballot Measure No. 1 � 15PFVR            \
Statistic                                             NO       YES   
Party                                                 NP        NP   
Precinct                                                             
01-446 Aurora                                   0.334511  0.665489   
01-455 Fairbanks No. 1                          0.316327  0.683673   
01-465 Fairbanks No. 2                          0.384615  0.615385   
01-470 Fairbanks No. 3                          0.336714  0.663286   
01-475 Fairbanks No. 4                          0.397059  0.602941   
01-480 Fairbanks No. 5                          0.322767  0.677233   
01-485 Fairbanks No. 6                          0.355114  0.644886   
01-490 Fairbanks No. 7                          0.326531  0.673469   
01-495 Fairbanks No. 10                         0.408163  0.591837   
02-345 Badger No. 2                             0.405784  0.594216   
02-355 Fairbanks No. 8                          0.320883  0.679117   
02-365 Fairbanks No. 9                          0.348195  0.651805   
02-375 Fort Wainwright                          0.198830  0.801170   
03-130 Badger No. 1                             0.416981  0.583019   
03-135 Chena Lakes                              0.397397  0.602603   
03-165 Newby                                    0.351505  0.648495   
03-175 North Pole                               0.377816  0.622184   
03-183 Plack                                    0.415385  0.584615   
04-230 Ester                                    0.306288  0.693712   
04-240 Farmers Loop                             0.294746  0.705254   
04-250 Goldstream No. 1                         0.309572  0.690428   
04-260 Goldstream No. 2                         0.297814  0.702186   
04-265 Steese East-Gilmore                      0.382485  0.617515   
04-270 Steese West                              0.378904  0.621096   
04-280 University Hills                         0.245192  0.754808   
05-580 Airport                                  0.387097  0.612903   
05-582 Chena                                    0.305484  0.694516   
05-586 Geist                                    0.326121  0.673879   
05-587 Lakeview                                 0.477941  0.522059   
05-588 Pike                                     0.423729  0.576271   
...                                                  ...       ...   
District 37 - Early Voting                      0.292929  0.707071   
District 37 - Question                          0.291457  0.708543   
District 38 - Absentee                          0.239300  0.760700   
District 38 - Early Voting                      0.268293  0.731707   
District 38 - Question                          0.327103  0.672897   
District 39 - Absentee                          0.224189  0.775811   
District 39 - Early Voting                      0.232628  0.767372   
District 39 - Question                          0.293269  0.706731   
District 4 - Absentee                           0.262219  0.737781   
District 4 - Early Voting                       0.260328  0.739672   
District 4 - Question                           0.343915  0.656085   
District 40 - Absentee                          0.235060  0.764940   
District 40 - Early Voting                      0.406780  0.593220   
District 40 - Question                          0.355649  0.644351   
District 5 - Absentee                           0.263069  0.736931   
District 5 - Early Voting                       0.266075  0.733925   
District 5 - Question                           0.305949  0.694051   
District 6 - Absentee                           0.348087  0.651913   
District 6 - Early Voting                       0.364070  0.635930   
District 6 - Question                           0.347578  0.652422   
District 7 - Absentee                           0.368324  0.631676   
District 7 - Early Voting                       0.392720  0.607280   
District 7

In [15]:
elections_dict['PRIM16']

Race                       HOUSE DISTRICT 1 (ADL) HOUSE DISTRICT 10 (ADL)  \
Statistic                      Kawasaki, Scott J.    Hartley, Christian M   
Party                                         DEM                     DEM   
Precinct                                                                    
01-446 Aurora                                 1.0                     NaN   
01-455 Fairbanks No. 1                        1.0                     NaN   
01-465 Fairbanks No. 2                        1.0                     NaN   
01-470 Fairbanks No. 3                        1.0                     NaN   
01-475 Fairbanks No. 4                        1.0                     NaN   
01-480 Fairbanks No. 5                        1.0                     NaN   
01-485 Fairbanks No. 6                        1.0                     NaN   
01-490 Fairbanks No. 7                        1.0                     NaN   
01-495 Fairbanks No. 10                       1.0                     NaN   
02-345 Badger No. 2                           NaN                     NaN   
02-355 Fairbanks No. 8                        NaN                     NaN   
02-365 Fairbanks No. 9                        NaN                     NaN   
02-375 Fort Wainwright                        NaN                     NaN   
03-130 Badger No. 1                           NaN                     NaN   
03-135 Chena Lakes                            NaN                     NaN   
03-165 Newby                                  NaN                     NaN   
03-175 North Pole                             NaN                     NaN   
03-183 Plack                                  NaN                     NaN   
04-230 Ester                                  NaN                     NaN   
04-240 Farmers Loop                           NaN                     NaN   
04-250 Goldstream No. 1                       NaN                     NaN   
04-260 Goldstream No. 2                       NaN                     NaN   
04-265 Steese East-Gilmore                    NaN                     NaN   
04-270 Steese West                            NaN                     NaN   
04-280 University Hills                       NaN                     NaN   
05-580 Airport                                NaN                     NaN   
05-582 Chena                                  NaN                     NaN   
05-586 Geist                                  NaN                     NaN   
05-587 Lakeview                               NaN                     NaN   
05-588 Pike                                   NaN                     NaN   
...                                           ...                     ...   
District 37 - Question                        NaN                     NaN   
District 38 - Absentee                        NaN                     NaN   
District 38 - Early Voting                    NaN                     NaN   
District 38 - Question                        NaN                     NaN   
District 39 - Absentee                        NaN                     NaN   
District 39 - Early Voting                    NaN                     NaN   
District 39 - Question                        NaN                     NaN   
District 4 - Absentee                         NaN                     NaN   
District 4 - Early Voting                     NaN                     NaN   
District 4 - Question                         NaN                     NaN   
District 40 - Absentee                        NaN                     NaN   
District 40 - Early Voting                    NaN                     NaN   
District 40 - Question                        NaN                     NaN   
District 5 - Absentee                         NaN                     NaN   
District 5 - Early Voting                     NaN                     NaN   
District 5 - Question                         NaN                     NaN   
District 6 - Absentee                         NaN                     NaN

In [16]:
elections_2014 = elections_dict['GENR14'].join(elections_dict['PRIM14'], lsuffix='_G', rsuffix='_P')

In [17]:
elections_2014

Race                       Ballot Measure 2 - 13PSUM            \
Statistic                                         NO       YES   
Party                                             NP        NP   
Precinct                                                         
01-446 Aurora                               0.452193  0.547807   
01-455 Fairbanks No. 1                      0.380208  0.619792   
01-465 Fairbanks No. 2                      0.339683  0.660317   
01-470 Fairbanks No. 3                      0.424547  0.575453   
01-475 Fairbanks No. 4                      0.387097  0.612903   
01-480 Fairbanks No. 5                      0.491826  0.508174   
01-485 Fairbanks No. 6                      0.386059  0.613941   
01-490 Fairbanks No. 7                      0.383621  0.616379   
01-495 Fairbanks No. 10                     0.473404  0.526596   
02-345 Badger No. 2                         0.409950  0.590050   
02-355 Fairbanks No. 8                      0.451557  0.548443   
02-365 Fairbanks No. 9                      0.489437  0.510563   
02-375 Fort Wainwright                      0.535714  0.464286   
03-130 Badger No. 1                         0.476699  0.523301   
03-135 Chena Lakes                          0.539859  0.460141   
03-165 Newby                                0.516493  0.483507   
03-175 North Pole                           0.546490  0.453510   
03-183 Plack                                0.504193  0.495807   
04-230 Ester                                0.355263  0.644737   
04-240 Farmers Loop                         0.420635  0.579365   
04-250 Goldstream No. 1                     0.289417  0.710583   
04-260 Goldstream No. 2                     0.293463  0.706537   
04-265 Steese East-Gilmore                  0.488424  0.511576   
04-270 Steese West                          0.440578  0.559422   
04-280 University Hills                     0.304147  0.695853   
05-580 Airport                              0.454545  0.545455   
05-582 Chena                                0.381941  0.618059   
05-586 Geist                                0.446532  0.553468   
05-587 Lakeview                             0.380282  0.619718   
05-588 Pike                                 0.385650  0.614350   
...                                              ...       ...   
District 37 - Absentee                      0.417582  0.582418   
District 37 - Early Voting                  0.368421  0.631579   
District 37 - Question                      0.332031  0.667969   
District 38 - Absentee                      0.531250  0.468750   
District 38 - Early Voting                  0.482759  0.517241   
District 38 - Question                      0.489362  0.510638   
District 39 - Absentee                      0.454955  0.545045   
District 39 - Early Voting                  0.368231  0.631769   
District 39 - Question                      0.417431  0.582569   
District 4 - Absentee                       0.359223  0.640777   
District 4 - Early Voting                   0.363128  0.636872   
District 4 - Question                       0.283117  0.716883   
District 40 - Absentee                      0.405882  0.594118   
District 40 - Early Voting                  0.558824  0.441176   
District 40 - Question                      0.316327  0.683673   
District 5 - Absentee                       0.356077  0.643923   
District 5 - Early Voting                   0.374289  0.625711   
District 5 - Question                       0.273810  0.726190   
District 6 - Absentee                       0.483208  0.516792   
District 6 - Early Voting                   0.394144  0.605856   
District 6 - Question                       0.355140  0.644860   
District 7 - Absentee                       0.533502  0.466498   
District 7 - Early Voting                   0.573218  0.426782   
District 7 - Question                       0.407332  0.592668   
District 8 - Absentee                       0.536649  0.463351   
District 8 - Early Voting               

In [18]:
elections_2016 = elections_dict['GENR16'].join(elections_dict['PRIM16'], lsuffix='_G', rsuffix='_P')

In [19]:
elections_2016

Race                       Ballot Measure No. 1 � 15PFVR            \
Statistic                                             NO       YES   
Party                                                 NP        NP   
Precinct                                                             
01-446 Aurora                                   0.334511  0.665489   
01-455 Fairbanks No. 1                          0.316327  0.683673   
01-465 Fairbanks No. 2                          0.384615  0.615385   
01-470 Fairbanks No. 3                          0.336714  0.663286   
01-475 Fairbanks No. 4                          0.397059  0.602941   
01-480 Fairbanks No. 5                          0.322767  0.677233   
01-485 Fairbanks No. 6                          0.355114  0.644886   
01-490 Fairbanks No. 7                          0.326531  0.673469   
01-495 Fairbanks No. 10                         0.408163  0.591837   
02-345 Badger No. 2                             0.405784  0.594216   
02-355 Fairbanks No. 8                          0.320883  0.679117   
02-365 Fairbanks No. 9                          0.348195  0.651805   
02-375 Fort Wainwright                          0.198830  0.801170   
03-130 Badger No. 1                             0.416981  0.583019   
03-135 Chena Lakes                              0.397397  0.602603   
03-165 Newby                                    0.351505  0.648495   
03-175 North Pole                               0.377816  0.622184   
03-183 Plack                                    0.415385  0.584615   
04-230 Ester                                    0.306288  0.693712   
04-240 Farmers Loop                             0.294746  0.705254   
04-250 Goldstream No. 1                         0.309572  0.690428   
04-260 Goldstream No. 2                         0.297814  0.702186   
04-265 Steese East-Gilmore                      0.382485  0.617515   
04-270 Steese West                              0.378904  0.621096   
04-280 University Hills                         0.245192  0.754808   
05-580 Airport                                  0.387097  0.612903   
05-582 Chena                                    0.305484  0.694516   
05-586 Geist                                    0.326121  0.673879   
05-587 Lakeview                                 0.477941  0.522059   
05-588 Pike                                     0.423729  0.576271   
...                                                  ...       ...   
District 37 - Early Voting                      0.292929  0.707071   
District 37 - Question                          0.291457  0.708543   
District 38 - Absentee                          0.239300  0.760700   
District 38 - Early Voting                      0.268293  0.731707   
District 38 - Question                          0.327103  0.672897   
District 39 - Absentee                          0.224189  0.775811   
District 39 - Early Voting                      0.232628  0.767372   
District 39 - Question                          0.293269  0.706731   
District 4 - Absentee                           0.262219  0.737781   
District 4 - Early Voting                       0.260328  0.739672   
District 4 - Question                           0.343915  0.656085   
District 40 - Absentee                          0.235060  0.764940   
District 40 - Early Voting                      0.406780  0.593220   
District 40 - Question                          0.355649  0.644351   
District 5 - Absentee                           0.263069  0.736931   
District 5 - Early Voting                       0.266075  0.733925   
District 5 - Question                           0.305949  0.694051   
District 6 - Absentee                           0.348087  0.651913   
District 6 - Early Voting                       0.364070  0.635930   
District 6 - Question                           0.347578  0.652422   
District 7 - Absentee                           0.368324  0.631676   
District 7 - Early Voting                       0.392720  0.607280   
District 7

In [20]:
elections_2014_2016 = elections_2014.join(elections_2016, lsuffix='_2014', rsuffix='_2016')

In [21]:
elections_2014_2016

Race                       Ballot Measure 2 - 13PSUM            \
Statistic                                         NO       YES   
Party                                             NP        NP   
Precinct                                                         
01-446 Aurora                               0.452193  0.547807   
01-455 Fairbanks No. 1                      0.380208  0.619792   
01-465 Fairbanks No. 2                      0.339683  0.660317   
01-470 Fairbanks No. 3                      0.424547  0.575453   
01-475 Fairbanks No. 4                      0.387097  0.612903   
01-480 Fairbanks No. 5                      0.491826  0.508174   
01-485 Fairbanks No. 6                      0.386059  0.613941   
01-490 Fairbanks No. 7                      0.383621  0.616379   
01-495 Fairbanks No. 10                     0.473404  0.526596   
02-345 Badger No. 2                         0.409950  0.590050   
02-355 Fairbanks No. 8                      0.451557  0.548443   
02-365 Fairbanks No. 9                      0.489437  0.510563   
02-375 Fort Wainwright                      0.535714  0.464286   
03-130 Badger No. 1                         0.476699  0.523301   
03-135 Chena Lakes                          0.539859  0.460141   
03-165 Newby                                0.516493  0.483507   
03-175 North Pole                           0.546490  0.453510   
03-183 Plack                                0.504193  0.495807   
04-230 Ester                                0.355263  0.644737   
04-240 Farmers Loop                         0.420635  0.579365   
04-250 Goldstream No. 1                     0.289417  0.710583   
04-260 Goldstream No. 2                     0.293463  0.706537   
04-265 Steese East-Gilmore                  0.488424  0.511576   
04-270 Steese West                          0.440578  0.559422   
04-280 University Hills                     0.304147  0.695853   
05-580 Airport                              0.454545  0.545455   
05-582 Chena                                0.381941  0.618059   
05-586 Geist                                0.446532  0.553468   
05-587 Lakeview                             0.380282  0.619718   
05-588 Pike                                 0.385650  0.614350   
...                                              ...       ...   
District 37 - Absentee                      0.417582  0.582418   
District 37 - Early Voting                  0.368421  0.631579   
District 37 - Question                      0.332031  0.667969   
District 38 - Absentee                      0.531250  0.468750   
District 38 - Early Voting                  0.482759  0.517241   
District 38 - Question                      0.489362  0.510638   
District 39 - Absentee                      0.454955  0.545045   
District 39 - Early Voting                  0.368231  0.631769   
District 39 - Question                      0.417431  0.582569   
District 4 - Absentee                       0.359223  0.640777   
District 4 - Early Voting                   0.363128  0.636872   
District 4 - Question                       0.283117  0.716883   
District 40 - Absentee                      0.405882  0.594118   
District 40 - Early Voting                  0.558824  0.441176   
District 40 - Question                      0.316327  0.683673   
District 5 - Absentee                       0.356077  0.643923   
District 5 - Early Voting                   0.374289  0.625711   
District 5 - Question                       0.273810  0.726190   
District 6 - Absentee                       0.483208  0.516792   
District 6 - Early Voting                   0.394144  0.605856   
District 6 - Question                       0.355140  0.644860   
District 7 - Absentee                       0.533502  0.466498   
District 7 - Early Voting                   0.573218  0.426782   
District 7 - Question                       0.407332  0.592668   
District 8 - Absentee                       0.536649  0.463351   
District 8 - Early Voting               

In [22]:
elections_2014_2016.to_csv("elections_2014_2016.csv")

In [ ]:
#Below lines are for testing aspects of data_clean function

In [12]:
GENR14copy = GENR14.copy()
GENR14copy.head()

,0,1,2,3,4,5,6
0,01-446 Aurora,Race Statistics,Number of Precincts,NP,Total,1.0,NaN
1,01-446 Aurora,Race Statistics,Number of Precincts Reporting,NP,Total,1.0,NaN
2,01-446 Aurora,UNITED STATES SENATOR,Number of Precincts for Race,NP,Total,1.0,NaN
3,01-446 Aurora,UNITED STATES SENATOR,Number of Precincts Reporting,NP,Total,1.0,NaN
4,01-446 Aurora,UNITED STATES SENATOR,Registered Voters,NP,Total,2426.0,NaN


In [13]:
GENR14copy.drop([4, 6], axis=1, inplace=True)
GENR14copy.head()

,0,1,2,3,5
0,01-446 Aurora,Race Statistics,Number of Precincts,NP,1.0
1,01-446 Aurora,Race Statistics,Number of Precincts Reporting,NP,1.0
2,01-446 Aurora,UNITED STATES SENATOR,Number of Precincts for Race,NP,1.0
3,01-446 Aurora,UNITED STATES SENATOR,Number of Precincts Reporting,NP,1.0
4,01-446 Aurora,UNITED STATES SENATOR,Registered Voters,NP,2426.0


In [14]:
GENR14copy.shape

(56184, 5)

In [15]:
GENR14copy.head(20)

,0,1,2,3,5
0,01-446 Aurora,Race Statistics,Number of Precincts,NP,1.0
1,01-446 Aurora,Race Statistics,Number of Precincts Reporting,NP,1.0
2,01-446 Aurora,UNITED STATES SENATOR,Number of Precincts for Race,NP,1.0
3,01-446 Aurora,UNITED STATES SENATOR,Number of Precincts Reporting,NP,1.0
4,01-446 Aurora,UNITED STATES SENATOR,Registered Voters,NP,2426.0
5,01-446 Aurora,UNITED STATES SENATOR,Times Counted,NP,906.0
6,01-446 Aurora,UNITED STATES SENATOR,"Begich, Mark",DEM,413.0
7,01-446 Aurora,UNITED STATES SENATOR,"Fish, Mark S.",LIB,30.0
8,01-446 Aurora,UNITED STATES SENATOR,"Gianoutsos, Ted",NA,22.0
9,01-446 Aurora,UNITED STATES SENATOR,"Sullivan, Dan",REP,434.0


In [16]:
GENR14copy.columns = ['Precinct', 'Race', 'Statistic', 'Party', 'Count']
GENR14copy.head(20)

,Precinct,Race,Statistic,Party,Count
0,01-446 Aurora,Race Statistics,Number of Precincts,NP,1.0
1,01-446 Aurora,Race Statistics,Number of Precincts Reporting,NP,1.0
2,01-446 Aurora,UNITED STATES SENATOR,Number of Precincts for Race,NP,1.0
3,01-446 Aurora,UNITED STATES SENATOR,Number of Precincts Reporting,NP,1.0
4,01-446 Aurora,UNITED STATES SENATOR,Registered Voters,NP,2426.0
5,01-446 Aurora,UNITED STATES SENATOR,Times Counted,NP,906.0
6,01-446 Aurora,UNITED STATES SENATOR,"Begich, Mark",DEM,413.0
7,01-446 Aurora,UNITED STATES SENATOR,"Fish, Mark S.",LIB,30.0
8,01-446 Aurora,UNITED STATES SENATOR,"Gianoutsos, Ted",NA,22.0
9,01-446 Aurora,UNITED STATES SENATOR,"Sullivan, Dan",REP,434.0


In [17]:
for c in ['Precinct', 'Race', 'Statistic', 'Party']:
    GENR14copy[c] = pd.core.strings.str_strip(GENR14copy[c])

GENR14copy.head(20)

,Precinct,Race,Statistic,Party,Count
0,01-446 Aurora,Race Statistics,Number of Precincts,NP,1.0
1,01-446 Aurora,Race Statistics,Number of Precincts Reporting,NP,1.0
2,01-446 Aurora,UNITED STATES SENATOR,Number of Precincts for Race,NP,1.0
3,01-446 Aurora,UNITED STATES SENATOR,Number of Precincts Reporting,NP,1.0
4,01-446 Aurora,UNITED STATES SENATOR,Registered Voters,NP,2426.0
5,01-446 Aurora,UNITED STATES SENATOR,Times Counted,NP,906.0
6,01-446 Aurora,UNITED STATES SENATOR,"Begich, Mark",DEM,413.0
7,01-446 Aurora,UNITED STATES SENATOR,"Fish, Mark S.",LIB,30.0
8,01-446 Aurora,UNITED STATES SENATOR,"Gianoutsos, Ted",NA,22.0
9,01-446 Aurora,UNITED STATES SENATOR,"Sullivan, Dan",REP,434.0


In [19]:
GENR14copy.Statistic = GENR14copy.Statistic.replace("Write-in.*", "Write-in", regex=True)

GENR14copy.head(20)

,Precinct,Race,Statistic,Party,Count
0,01-446 Aurora,Race Statistics,Number of Precincts,NP,1.0
1,01-446 Aurora,Race Statistics,Number of Precincts Reporting,NP,1.0
2,01-446 Aurora,UNITED STATES SENATOR,Number of Precincts for Race,NP,1.0
3,01-446 Aurora,UNITED STATES SENATOR,Number of Precincts Reporting,NP,1.0
4,01-446 Aurora,UNITED STATES SENATOR,Registered Voters,NP,2426.0
5,01-446 Aurora,UNITED STATES SENATOR,Times Counted,NP,906.0
6,01-446 Aurora,UNITED STATES SENATOR,"Begich, Mark",DEM,413.0
7,01-446 Aurora,UNITED STATES SENATOR,"Fish, Mark S.",LIB,30.0
8,01-446 Aurora,UNITED STATES SENATOR,"Gianoutsos, Ted",NA,22.0
9,01-446 Aurora,UNITED STATES SENATOR,"Sullivan, Dan",REP,434.0


In [20]:
#Try to drop "Race Statistics" rows
GENR14copy = GENR14copy[GENR14copy.Race != "Race Statistics"]
GENR14copy.head(20)

,Precinct,Race,Statistic,Party,Count
2,01-446 Aurora,UNITED STATES SENATOR,Number of Precincts for Race,NP,1.0
3,01-446 Aurora,UNITED STATES SENATOR,Number of Precincts Reporting,NP,1.0
4,01-446 Aurora,UNITED STATES SENATOR,Registered Voters,NP,2426.0
5,01-446 Aurora,UNITED STATES SENATOR,Times Counted,NP,906.0
6,01-446 Aurora,UNITED STATES SENATOR,"Begich, Mark",DEM,413.0
7,01-446 Aurora,UNITED STATES SENATOR,"Fish, Mark S.",LIB,30.0
8,01-446 Aurora,UNITED STATES SENATOR,"Gianoutsos, Ted",NA,22.0
9,01-446 Aurora,UNITED STATES SENATOR,"Sullivan, Dan",REP,434.0
10,01-446 Aurora,UNITED STATES SENATOR,Write-in,NP,2.0
11,01-446 Aurora,US REPRESENTATIVE,Number of Precincts for Race,NP,1.0


In [21]:
GENR14copy.shape

(55062, 5)

In [22]:
56184 - 55062

1122

In [23]:
GENR14pivot = pd.pivot_table(GENR14copy, values='Count', columns=['Race', 'Statistic', 'Party'], index='Precinct')
GENR14pivot

Race                       Ballot Measure 2 - 13PSUM  \
Statistic                                         NO   
Party                                             NP   
Precinct                                               
01-446 Aurora                                  402.0   
01-455 Fairbanks No. 1                          73.0   
01-465 Fairbanks No. 2                         107.0   
01-470 Fairbanks No. 3                         211.0   
01-475 Fairbanks No. 4                         108.0   
01-480 Fairbanks No. 5                         361.0   
01-485 Fairbanks No. 6                         144.0   
01-490 Fairbanks No. 7                         178.0   
01-495 Fairbanks No. 10                         89.0   
02-345 Badger No. 2                            412.0   
02-355 Fairbanks No. 8                         261.0   
02-365 Fairbanks No. 9                         417.0   
02-375 Fort Wainwright                         105.0   
03-130 Badger No. 1                            491.0   
03-135 Chena Lakes                             535.0   
03-165 Newby                                   595.0   
03-175 North Pole                              288.0   
03-183 Plack                                   481.0   
04-230 Ester                                   162.0   
04-240 Farmers Loop                            477.0   
04-250 Goldstream No. 1                        134.0   
04-260 Goldstream No. 2                        211.0   
04-265 Steese East-Gilmore                     654.0   
04-270 Steese West                             671.0   
04-280 University Hills                         66.0   
05-580 Airport                                  70.0   
05-582 Chena                                   736.0   
05-586 Geist                                   309.0   
05-587 Lakeview                                 54.0   
05-588 Pike                                     86.0   
...                                              ...   
District 37 - Absentee                         494.0   
District 37 - Early Voting                      28.0   
District 37 - Question                          85.0   
District 38 - Absentee                         289.0   
District 38 - Early Voting                      14.0   
District 38 - Question                         184.0   
District 39 - Absentee                         202.0   
District 39 - Early Voting                     102.0   
District 39 - Question                          91.0   
District 4 - Absentee                          407.0   
District 4 - Early Voting                      455.0   
District 4 - Question                          109.0   
District 40 - Absentee                         207.0   
District 40 - Early Voting                      19.0   
District 40 - Question                          62.0   
District 5 - Absentee                          334.0   
District 5 - Early Voting                      329.0   
District 5 - Question                           92.0   
District 6 - Absentee                          705.0   
District 6 - Early Voting                      175.0   
District 6 - Question                          114.0   
District 7 - Absentee                          422.0   
District 7 - Early Voting                      732.0   
District 7 - Question                          200.0   
District 8 - Absentee                          410.0   
District 8 - Early Voting                      396.0   
District 8 - Question                          117.0   
District 9 - Absentee                          704.0   
District 9 - Early Voting                      202.0   
District 9 - Question                          128.0   

Race                                                      \
Statistic                  Number of Precincts Reporting   
Party                                                 NP   
Precinct                                                   
01-446 Aurora                                        1.0   
01-455 Fairbanks No. 1                               1.0   

In [24]:
GENR14pivot.shape

(561, 502)

In [25]:
# Try to sum total votes per precinct in each race
GENR14pivot.columns

MultiIndex(levels=[[u'Ballot Measure 2 - 13PSUM', u'Ballot Measure 3 - 13MINW', u'Ballot Measure 4 - 12BBAY', u'Dis Court JD1 Miller', u'Dis Court JD3 Chung', u'Dis Court JD3 Clark', u'Dis Court JD3 Estelle', u'Dis Court JD3 Illsley', u'Dis Court JD3 Motyka', u'Dis Court JD3 Rhoades', u'Dis Court JD3 Wolfe', u'Dis Court JD4 Seekins', u'GOVERNOR/LT GOVERNOR', u'HOUSE DISTRICT 1', u'HOUSE DISTRICT 10', u'HOUSE DISTRICT 11', u'HOUSE DISTRICT 12', u'HOUSE DISTRICT 13', u'HOUSE DISTRICT 14', u'HOUSE DISTRICT 15', u'HOUSE DISTRICT 16', u'HOUSE DISTRICT 17', u'HOUSE DISTRICT 18', u'HOUSE DISTRICT 19', u'HOUSE DISTRICT 2', u'HOUSE DISTRICT 20', u'HOUSE DISTRICT 21', u'HOUSE DISTRICT 22', u'HOUSE DISTRICT 23', u'HOUSE DISTRICT 24', u'HOUSE DISTRICT 25', u'HOUSE DISTRICT 26', u'HOUSE DISTRICT 27', u'HOUSE DISTRICT 28', u'HOUSE DISTRICT 29', u'HOUSE DISTRICT 3', u'HOUSE DISTRICT 30', u'HOUSE DISTRICT 31', u'HOUSE DISTRICT 32', u'HOUSE DISTRICT 33', u'HOUSE DISTRICT 34', u'HOUSE DISTRICT 35', u'HO

In [26]:
GENR14pivot.columns.values

array([('Ballot Measure 2 - 13PSUM', 'NO', 'NP'),
       ('Ballot Measure 2 - 13PSUM', 'Number of Precincts Reporting', 'NP'),
       ('Ballot Measure 2 - 13PSUM', 'Number of Precincts for Race', 'NP'),
       ('Ballot Measure 2 - 13PSUM', 'Registered Voters', 'NP'),
       ('Ballot Measure 2 - 13PSUM', 'Times Counted', 'NP'),
       ('Ballot Measure 2 - 13PSUM', 'YES', 'NP'),
       ('Ballot Measure 3 - 13MINW', 'NO', 'NP'),
       ('Ballot Measure 3 - 13MINW', 'Number of Precincts Reporting', 'NP'),
       ('Ballot Measure 3 - 13MINW', 'Number of Precincts for Race', 'NP'),
       ('Ballot Measure 3 - 13MINW', 'Registered Voters', 'NP'),
       ('Ballot Measure 3 - 13MINW', 'Times Counted', 'NP'),
       ('Ballot Measure 3 - 13MINW', 'YES', 'NP'),
       ('Ballot Measure 4 - 12BBAY', 'NO', 'NP'),
       ('Ballot Measure 4 - 12BBAY', 'Number of Precincts Reporting', 'NP'),
       ('Ballot Measure 4 - 12BBAY', 'Number of Precincts for Race', 'NP'),
       ('Ballot Measure 4 - 12BBAY', 

In [27]:
GENR14pivot.groupby(level=0, axis=1).sum()

Race,Ballot Measure 2 - 13PSUM,Ballot Measure 3 - 13MINW,Ballot Measure 4 - 12BBAY,Dis Court JD1 Miller,Dis Court JD3 Chung,Dis Court JD3 Clark,Dis Court JD3 Estelle,Dis Court JD3 Illsley,Dis Court JD3 Motyka,Dis Court JD3 Rhoades,...,SENATE DISTRICT Q,SENATE DISTRICT S,SENATE DISTRICT T,Sup Court JD1 Menendez,Sup Court JD2 Roetman,Sup Court JD3 Guidi,Sup Court JD3 Miller,Supreme Crt-Justice Stowers,UNITED STATES SENATOR,US REPRESENTATIVE
Precinct,,,,,,,,,,,,,,,,,,,,,
01-446 Aurora,4223.0,4229.0,4202.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4125.0,4235.0,4223.0
01-455 Fairbanks No. 1,888.0,891.0,889.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,876.0,892.0,891.0
01-465 Fairbanks No. 2,1774.0,1777.0,1775.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1746.0,1786.0,1780.0
01-470 Fairbanks No. 3,2389.0,2399.0,2392.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2352.0,2401.0,2397.0
01-475 Fairbanks No. 4,1299.0,1300.0,1294.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1272.0,1304.0,1301.0
01-480 Fairbanks No. 5,3456.0,3454.0,3430.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3384.0,3461.0,3453.0
01-485 Fairbanks No. 6,2159.0,2169.0,2165.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2140.0,2171.0,2169.0
01-490 Fairbanks No. 7,2793.0,2802.0,2794.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2757.0,2809.0,2804.0
01-495 Fairbanks No. 10,1163.0,1162.0,1159.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1142.0,1165.0,1162.0


In [29]:
GENR14pivot.sum(level=0, axis=1)

Race,Ballot Measure 2 - 13PSUM,Ballot Measure 3 - 13MINW,Ballot Measure 4 - 12BBAY,Dis Court JD1 Miller,Dis Court JD3 Chung,Dis Court JD3 Clark,Dis Court JD3 Estelle,Dis Court JD3 Illsley,Dis Court JD3 Motyka,Dis Court JD3 Rhoades,...,SENATE DISTRICT Q,SENATE DISTRICT S,SENATE DISTRICT T,Sup Court JD1 Menendez,Sup Court JD2 Roetman,Sup Court JD3 Guidi,Sup Court JD3 Miller,Supreme Crt-Justice Stowers,UNITED STATES SENATOR,US REPRESENTATIVE
Precinct,,,,,,,,,,,,,,,,,,,,,
01-446 Aurora,4223.0,4229.0,4202.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4125.0,4235.0,4223.0
01-455 Fairbanks No. 1,888.0,891.0,889.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,876.0,892.0,891.0
01-465 Fairbanks No. 2,1774.0,1777.0,1775.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1746.0,1786.0,1780.0
01-470 Fairbanks No. 3,2389.0,2399.0,2392.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2352.0,2401.0,2397.0
01-475 Fairbanks No. 4,1299.0,1300.0,1294.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1272.0,1304.0,1301.0
01-480 Fairbanks No. 5,3456.0,3454.0,3430.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3384.0,3461.0,3453.0
01-485 Fairbanks No. 6,2159.0,2169.0,2165.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2140.0,2171.0,2169.0
01-490 Fairbanks No. 7,2793.0,2802.0,2794.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2757.0,2809.0,2804.0
01-495 Fairbanks No. 10,1163.0,1162.0,1159.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1142.0,1165.0,1162.0


In [33]:
GENR14pivot.drop(["Number of Precincts Reporting"], level=1, axis=1)

Race                       Ballot Measure 2 - 13PSUM  \
Statistic                                         NO   
Party                                             NP   
Precinct                                               
01-446 Aurora                                  402.0   
01-455 Fairbanks No. 1                          73.0   
01-465 Fairbanks No. 2                         107.0   
01-470 Fairbanks No. 3                         211.0   
01-475 Fairbanks No. 4                         108.0   
01-480 Fairbanks No. 5                         361.0   
01-485 Fairbanks No. 6                         144.0   
01-490 Fairbanks No. 7                         178.0   
01-495 Fairbanks No. 10                         89.0   
02-345 Badger No. 2                            412.0   
02-355 Fairbanks No. 8                         261.0   
02-365 Fairbanks No. 9                         417.0   
02-375 Fort Wainwright                         105.0   
03-130 Badger No. 1                            491.0   
03-135 Chena Lakes                             535.0   
03-165 Newby                                   595.0   
03-175 North Pole                              288.0   
03-183 Plack                                   481.0   
04-230 Ester                                   162.0   
04-240 Farmers Loop                            477.0   
04-250 Goldstream No. 1                        134.0   
04-260 Goldstream No. 2                        211.0   
04-265 Steese East-Gilmore                     654.0   
04-270 Steese West                             671.0   
04-280 University Hills                         66.0   
05-580 Airport                                  70.0   
05-582 Chena                                   736.0   
05-586 Geist                                   309.0   
05-587 Lakeview                                 54.0   
05-588 Pike                                     86.0   
...                                              ...   
District 37 - Absentee                         494.0   
District 37 - Early Voting                      28.0   
District 37 - Question                          85.0   
District 38 - Absentee                         289.0   
District 38 - Early Voting                      14.0   
District 38 - Question                         184.0   
District 39 - Absentee                         202.0   
District 39 - Early Voting                     102.0   
District 39 - Question                          91.0   
District 4 - Absentee                          407.0   
District 4 - Early Voting                      455.0   
District 4 - Question                          109.0   
District 40 - Absentee                         207.0   
District 40 - Early Voting                      19.0   
District 40 - Question                          62.0   
District 5 - Absentee                          334.0   
District 5 - Early Voting                      329.0   
District 5 - Question                           92.0   
District 6 - Absentee                          705.0   
District 6 - Early Voting                      175.0   
District 6 - Question                          114.0   
District 7 - Absentee                          422.0   
District 7 - Early Voting                      732.0   
District 7 - Question                          200.0   
District 8 - Absentee                          410.0   
District 8 - Early Voting                      396.0   
District 8 - Question                          117.0   
District 9 - Absentee                          704.0   
District 9 - Early Voting                      202.0   
District 9 - Question                          128.0   

Race                                                                       \
Statistic                  Number of Precincts for Race Registered Voters   
Party                                                NP                NP   
Precinct                                                                    
01-446 Aurora                                      

In [34]:
GENR14pivot

Race                       Ballot Measure 2 - 13PSUM  \
Statistic                                         NO   
Party                                             NP   
Precinct                                               
01-446 Aurora                                  402.0   
01-455 Fairbanks No. 1                          73.0   
01-465 Fairbanks No. 2                         107.0   
01-470 Fairbanks No. 3                         211.0   
01-475 Fairbanks No. 4                         108.0   
01-480 Fairbanks No. 5                         361.0   
01-485 Fairbanks No. 6                         144.0   
01-490 Fairbanks No. 7                         178.0   
01-495 Fairbanks No. 10                         89.0   
02-345 Badger No. 2                            412.0   
02-355 Fairbanks No. 8                         261.0   
02-365 Fairbanks No. 9                         417.0   
02-375 Fort Wainwright                         105.0   
03-130 Badger No. 1                            491.0   
03-135 Chena Lakes                             535.0   
03-165 Newby                                   595.0   
03-175 North Pole                              288.0   
03-183 Plack                                   481.0   
04-230 Ester                                   162.0   
04-240 Farmers Loop                            477.0   
04-250 Goldstream No. 1                        134.0   
04-260 Goldstream No. 2                        211.0   
04-265 Steese East-Gilmore                     654.0   
04-270 Steese West                             671.0   
04-280 University Hills                         66.0   
05-580 Airport                                  70.0   
05-582 Chena                                   736.0   
05-586 Geist                                   309.0   
05-587 Lakeview                                 54.0   
05-588 Pike                                     86.0   
...                                              ...   
District 37 - Absentee                         494.0   
District 37 - Early Voting                      28.0   
District 37 - Question                          85.0   
District 38 - Absentee                         289.0   
District 38 - Early Voting                      14.0   
District 38 - Question                         184.0   
District 39 - Absentee                         202.0   
District 39 - Early Voting                     102.0   
District 39 - Question                          91.0   
District 4 - Absentee                          407.0   
District 4 - Early Voting                      455.0   
District 4 - Question                          109.0   
District 40 - Absentee                         207.0   
District 40 - Early Voting                      19.0   
District 40 - Question                          62.0   
District 5 - Absentee                          334.0   
District 5 - Early Voting                      329.0   
District 5 - Question                           92.0   
District 6 - Absentee                          705.0   
District 6 - Early Voting                      175.0   
District 6 - Question                          114.0   
District 7 - Absentee                          422.0   
District 7 - Early Voting                      732.0   
District 7 - Question                          200.0   
District 8 - Absentee                          410.0   
District 8 - Early Voting                      396.0   
District 8 - Question                          117.0   
District 9 - Absentee                          704.0   
District 9 - Early Voting                      202.0   
District 9 - Question                          128.0   

Race                                                      \
Statistic                  Number of Precincts Reporting   
Party                                                 NP   
Precinct                                                   
01-446 Aurora                                        1.0   
01-455 Fairbanks No. 1                               1.0   

In [35]:
GENR14votes = GENR14pivot.drop(["Number of Precincts for Race",
                                 "Number of Precincts Reporting",
                                 "Registered Voters",
                                 "Times Counted"], axis=1, level=1)
GENR14votes

Race                       Ballot Measure 2 - 13PSUM          \
Statistic                                         NO     YES   
Party                                             NP      NP   
Precinct                                                       
01-446 Aurora                                  402.0   487.0   
01-455 Fairbanks No. 1                          73.0   119.0   
01-465 Fairbanks No. 2                         107.0   208.0   
01-470 Fairbanks No. 3                         211.0   286.0   
01-475 Fairbanks No. 4                         108.0   171.0   
01-480 Fairbanks No. 5                         361.0   373.0   
01-485 Fairbanks No. 6                         144.0   229.0   
01-490 Fairbanks No. 7                         178.0   286.0   
01-495 Fairbanks No. 10                         89.0    99.0   
02-345 Badger No. 2                            412.0   593.0   
02-355 Fairbanks No. 8                         261.0   317.0   
02-365 Fairbanks No. 9                         417.0   435.0   
02-375 Fort Wainwright                         105.0    91.0   
03-130 Badger No. 1                            491.0   539.0   
03-135 Chena Lakes                             535.0   456.0   
03-165 Newby                                   595.0   557.0   
03-175 North Pole                              288.0   239.0   
03-183 Plack                                   481.0   473.0   
04-230 Ester                                   162.0   294.0   
04-240 Farmers Loop                            477.0   657.0   
04-250 Goldstream No. 1                        134.0   329.0   
04-260 Goldstream No. 2                        211.0   508.0   
04-265 Steese East-Gilmore                     654.0   685.0   
04-270 Steese West                             671.0   852.0   
04-280 University Hills                         66.0   151.0   
05-580 Airport                                  70.0    84.0   
05-582 Chena                                   736.0  1191.0   
05-586 Geist                                   309.0   383.0   
05-587 Lakeview                                 54.0    88.0   
05-588 Pike                                     86.0   137.0   
...                                              ...     ...   
District 37 - Absentee                         494.0   689.0   
District 37 - Early Voting                      28.0    48.0   
District 37 - Question                          85.0   171.0   
District 38 - Absentee                         289.0   255.0   
District 38 - Early Voting                      14.0    15.0   
District 38 - Question                         184.0   192.0   
District 39 - Absentee                         202.0   242.0   
District 39 - Early Voting                     102.0   175.0   
District 39 - Question                          91.0   127.0   
District 4 - Absentee                          407.0   726.0   
District 4 - Early Voting                      455.0   798.0   
District 4 - Question                          109.0   276.0   
District 40 - Absentee                         207.0   303.0   
District 40 - Early Voting                      19.0    15.0   
District 40 - Question                          62.0   134.0   
District 5 - Absentee                          334.0   604.0   
District 5 - Early Voting                      329.0   550.0   
District 5 - Question                           92.0   244.0   
District 6 - Absentee                          705.0   754.0   
District 6 - Early Voting                      175.0   269.0   
District 6 - Question                          114.0   207.0   
District 7 - Absentee                          422.0   369.0   
District 7 - Early Voting                      732.0   545.0   
District 7 - Question                          200.0   291.0   
District 8 - Absentee                          410.0   354.0   
District 8 - Early Voting                      396.0   349.0   
District 8 - Question                          117.0   249.0   
District 9 - Absentee           

In [36]:
GENR14votes.columns.levels

FrozenList([[u'Ballot Measure 2 - 13PSUM', u'Ballot Measure 3 - 13MINW', u'Ballot Measure 4 - 12BBAY', u'Dis Court JD1 Miller', u'Dis Court JD3 Chung', u'Dis Court JD3 Clark', u'Dis Court JD3 Estelle', u'Dis Court JD3 Illsley', u'Dis Court JD3 Motyka', u'Dis Court JD3 Rhoades', u'Dis Court JD3 Wolfe', u'Dis Court JD4 Seekins', u'GOVERNOR/LT GOVERNOR', u'HOUSE DISTRICT 1', u'HOUSE DISTRICT 10', u'HOUSE DISTRICT 11', u'HOUSE DISTRICT 12', u'HOUSE DISTRICT 13', u'HOUSE DISTRICT 14', u'HOUSE DISTRICT 15', u'HOUSE DISTRICT 16', u'HOUSE DISTRICT 17', u'HOUSE DISTRICT 18', u'HOUSE DISTRICT 19', u'HOUSE DISTRICT 2', u'HOUSE DISTRICT 20', u'HOUSE DISTRICT 21', u'HOUSE DISTRICT 22', u'HOUSE DISTRICT 23', u'HOUSE DISTRICT 24', u'HOUSE DISTRICT 25', u'HOUSE DISTRICT 26', u'HOUSE DISTRICT 27', u'HOUSE DISTRICT 28', u'HOUSE DISTRICT 29', u'HOUSE DISTRICT 3', u'HOUSE DISTRICT 30', u'HOUSE DISTRICT 31', u'HOUSE DISTRICT 32', u'HOUSE DISTRICT 33', u'HOUSE DISTRICT 34', u'HOUSE DISTRICT 35', u'HOUSE DIS

In [37]:
GENR14totals = GENR14votes.sum(level=0, axis=1)
GENR14totals

Race,Ballot Measure 2 - 13PSUM,Ballot Measure 3 - 13MINW,Ballot Measure 4 - 12BBAY,Dis Court JD1 Miller,Dis Court JD3 Chung,Dis Court JD3 Clark,Dis Court JD3 Estelle,Dis Court JD3 Illsley,Dis Court JD3 Motyka,Dis Court JD3 Rhoades,...,SENATE DISTRICT Q,SENATE DISTRICT S,SENATE DISTRICT T,Sup Court JD1 Menendez,Sup Court JD2 Roetman,Sup Court JD3 Guidi,Sup Court JD3 Miller,Supreme Crt-Justice Stowers,UNITED STATES SENATOR,US REPRESENTATIVE
Precinct,,,,,,,,,,,,,,,,,,,,,
01-446 Aurora,889.0,895.0,868.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,791.0,901.0,889.0
01-455 Fairbanks No. 1,192.0,195.0,193.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,180.0,196.0,195.0
01-465 Fairbanks No. 2,315.0,318.0,316.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,287.0,327.0,321.0
01-470 Fairbanks No. 3,497.0,507.0,500.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,460.0,509.0,505.0
01-475 Fairbanks No. 4,279.0,280.0,274.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,252.0,284.0,281.0
01-480 Fairbanks No. 5,734.0,732.0,708.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,662.0,739.0,731.0
01-485 Fairbanks No. 6,373.0,383.0,379.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,354.0,385.0,383.0
01-490 Fairbanks No. 7,464.0,473.0,465.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,428.0,480.0,475.0
01-495 Fairbanks No. 10,188.0,187.0,184.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,167.0,190.0,187.0


In [38]:
pd.merge(GENR14votes, GENR14totals, left_index=True, right_index=True)

C:\Users\Will\Anaconda2\lib\site-packages\pandas\tools\merge.py:489: UserWarning: merging between different levels can give an unintended result (3 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


,"(Ballot Measure 2 - 13PSUM, NO, NP)","(Ballot Measure 2 - 13PSUM, YES, NP)","(Ballot Measure 3 - 13MINW, NO, NP)","(Ballot Measure 3 - 13MINW, YES, NP)","(Ballot Measure 4 - 12BBAY, NO, NP)","(Ballot Measure 4 - 12BBAY, YES, NP)","(Dis Court JD1 Miller, NO, NP)","(Dis Court JD1 Miller, YES, NP)","(Dis Court JD3 Chung, NO, NP)","(Dis Court JD3 Chung, YES, NP)",...,SENATE DISTRICT Q,SENATE DISTRICT S,SENATE DISTRICT T,Sup Court JD1 Menendez,Sup Court JD2 Roetman,Sup Court JD3 Guidi,Sup Court JD3 Miller,Supreme Crt-Justice Stowers,UNITED STATES SENATOR,US REPRESENTATIVE
Precinct,,,,,,,,,,,,,,,,,,,,,
01-446 Aurora,402.0,487.0,246.0,649.0,304.0,564.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,791.0,901.0,889.0
01-455 Fairbanks No. 1,73.0,119.0,54.0,141.0,68.0,125.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,180.0,196.0,195.0
01-465 Fairbanks No. 2,107.0,208.0,71.0,247.0,92.0,224.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,287.0,327.0,321.0
01-470 Fairbanks No. 3,211.0,286.0,133.0,374.0,207.0,293.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,460.0,509.0,505.0
01-475 Fairbanks No. 4,108.0,171.0,74.0,206.0,105.0,169.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,252.0,284.0,281.0
01-480 Fairbanks No. 5,361.0,373.0,257.0,475.0,276.0,432.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,662.0,739.0,731.0
01-485 Fairbanks No. 6,144.0,229.0,104.0,279.0,144.0,235.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,354.0,385.0,383.0
01-490 Fairbanks No. 7,178.0,286.0,116.0,357.0,138.0,327.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,428.0,480.0,475.0
01-495 Fairbanks No. 10,89.0,99.0,49.0,138.0,75.0,109.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,167.0,190.0,187.0


In [39]:
GENR14totals.columns

AttributeError: 'Index' object has no attribute 'set_levels'

In [40]:
GENR14pct = GENR14votes / GENR14totals
GENR14pct

Race                       Ballot Measure 2 - 13PSUM            \
Statistic                                         NO       YES   
Party                                             NP        NP   
Precinct                                                         
01-446 Aurora                               0.452193  0.547807   
01-455 Fairbanks No. 1                      0.380208  0.619792   
01-465 Fairbanks No. 2                      0.339683  0.660317   
01-470 Fairbanks No. 3                      0.424547  0.575453   
01-475 Fairbanks No. 4                      0.387097  0.612903   
01-480 Fairbanks No. 5                      0.491826  0.508174   
01-485 Fairbanks No. 6                      0.386059  0.613941   
01-490 Fairbanks No. 7                      0.383621  0.616379   
01-495 Fairbanks No. 10                     0.473404  0.526596   
02-345 Badger No. 2                         0.409950  0.590050   
02-355 Fairbanks No. 8                      0.451557  0.548443   
02-365 Fairbanks No. 9                      0.489437  0.510563   
02-375 Fort Wainwright                      0.535714  0.464286   
03-130 Badger No. 1                         0.476699  0.523301   
03-135 Chena Lakes                          0.539859  0.460141   
03-165 Newby                                0.516493  0.483507   
03-175 North Pole                           0.546490  0.453510   
03-183 Plack                                0.504193  0.495807   
04-230 Ester                                0.355263  0.644737   
04-240 Farmers Loop                         0.420635  0.579365   
04-250 Goldstream No. 1                     0.289417  0.710583   
04-260 Goldstream No. 2                     0.293463  0.706537   
04-265 Steese East-Gilmore                  0.488424  0.511576   
04-270 Steese West                          0.440578  0.559422   
04-280 University Hills                     0.304147  0.695853   
05-580 Airport                              0.454545  0.545455   
05-582 Chena                                0.381941  0.618059   
05-586 Geist                                0.446532  0.553468   
05-587 Lakeview                             0.380282  0.619718   
05-588 Pike                                 0.385650  0.614350   
...                                              ...       ...   
District 37 - Absentee                      0.417582  0.582418   
District 37 - Early Voting                  0.368421  0.631579   
District 37 - Question                      0.332031  0.667969   
District 38 - Absentee                      0.531250  0.468750   
District 38 - Early Voting                  0.482759  0.517241   
District 38 - Question                      0.489362  0.510638   
District 39 - Absentee                      0.454955  0.545045   
District 39 - Early Voting                  0.368231  0.631769   
District 39 - Question                      0.417431  0.582569   
District 4 - Absentee                       0.359223  0.640777   
District 4 - Early Voting                   0.363128  0.636872   
District 4 - Question                       0.283117  0.716883   
District 40 - Absentee                      0.405882  0.594118   
District 40 - Early Voting                  0.558824  0.441176   
District 40 - Question                      0.316327  0.683673   
District 5 - Absentee                       0.356077  0.643923   
District 5 - Early Voting                   0.374289  0.625711   
District 5 - Question                       0.273810  0.726190   
District 6 - Absentee                       0.483208  0.516792   
District 6 - Early Voting                   0.394144  0.605856   
District 6 - Question                       0.355140  0.644860   
District 7 - Absentee                       0.533502  0.466498   
District 7 - Early Voting                   0.573218  0.426782   
District 7 - Question                       0.407332  0.592668   
District 8 - Absentee                       0.536649  0.463351   
District 8 - Early Voting               

In [41]:
#check percentages sum to 1
GENR14pct.sum(level=0, axis=1)

Race,Ballot Measure 2 - 13PSUM,Ballot Measure 3 - 13MINW,Ballot Measure 4 - 12BBAY,Dis Court JD1 Miller,Dis Court JD3 Chung,Dis Court JD3 Clark,Dis Court JD3 Estelle,Dis Court JD3 Illsley,Dis Court JD3 Motyka,Dis Court JD3 Rhoades,...,SENATE DISTRICT Q,SENATE DISTRICT S,SENATE DISTRICT T,Sup Court JD1 Menendez,Sup Court JD2 Roetman,Sup Court JD3 Guidi,Sup Court JD3 Miller,Supreme Crt-Justice Stowers,UNITED STATES SENATOR,US REPRESENTATIVE
Precinct,,,,,,,,,,,,,,,,,,,,,
01-446 Aurora,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0
01-455 Fairbanks No. 1,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0
01-465 Fairbanks No. 2,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0
01-470 Fairbanks No. 3,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0
01-475 Fairbanks No. 4,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0
01-480 Fairbanks No. 5,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0
01-485 Fairbanks No. 6,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0
01-490 Fairbanks No. 7,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0
01-495 Fairbanks No. 10,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0


In [46]:
GENR14turnout = GENR14pivot['US REPRESENTATIVE']["Times Counted"]["NP"] / GENR14pivot['US REPRESENTATIVE']["Registered Voters"]["NP"]
GENR14turnout

Precinct
01-446 Aurora                 0.373454
01-455 Fairbanks No. 1        0.402020
01-465 Fairbanks No. 2        0.290523
01-470 Fairbanks No. 3        0.374545
01-475 Fairbanks No. 4        0.388813
01-480 Fairbanks No. 5        0.377913
01-485 Fairbanks No. 6        0.277937
01-490 Fairbanks No. 7        0.263301
01-495 Fairbanks No. 10       0.247436
02-345 Badger No. 2           0.371171
02-355 Fairbanks No. 8        0.310309
02-365 Fairbanks No. 9        0.324991
02-375 Fort Wainwright        0.052632
03-130 Badger No. 1           0.359010
03-135 Chena Lakes            0.399048
03-165 Newby                  0.382951
03-175 North Pole             0.333126
03-183 Plack                  0.375289
04-230 Ester                  0.488818
04-240 Farmers Loop           0.448006
04-250 Goldstream No. 1       0.415128
04-260 Goldstream No. 2       0.517438
04-265 Steese East-Gilmore    0.423676
04-270 Steese West            0.390842
04-280 University Hills       0.400366
05-580 Airport  

In [54]:
GENR14final = GENR14votes.join(GENR14pct, lsuffix='', rsuffix='r')

GENR14final

Race                       Ballot Measure 2 - 13PSUM          \
Statistic                                         NO     YES   
Party                                             NP      NP   
Precinct                                                       
01-446 Aurora                                  402.0   487.0   
01-455 Fairbanks No. 1                          73.0   119.0   
01-465 Fairbanks No. 2                         107.0   208.0   
01-470 Fairbanks No. 3                         211.0   286.0   
01-475 Fairbanks No. 4                         108.0   171.0   
01-480 Fairbanks No. 5                         361.0   373.0   
01-485 Fairbanks No. 6                         144.0   229.0   
01-490 Fairbanks No. 7                         178.0   286.0   
01-495 Fairbanks No. 10                         89.0    99.0   
02-345 Badger No. 2                            412.0   593.0   
02-355 Fairbanks No. 8                         261.0   317.0   
02-365 Fairbanks No. 9                         417.0   435.0   
02-375 Fort Wainwright                         105.0    91.0   
03-130 Badger No. 1                            491.0   539.0   
03-135 Chena Lakes                             535.0   456.0   
03-165 Newby                                   595.0   557.0   
03-175 North Pole                              288.0   239.0   
03-183 Plack                                   481.0   473.0   
04-230 Ester                                   162.0   294.0   
04-240 Farmers Loop                            477.0   657.0   
04-250 Goldstream No. 1                        134.0   329.0   
04-260 Goldstream No. 2                        211.0   508.0   
04-265 Steese East-Gilmore                     654.0   685.0   
04-270 Steese West                             671.0   852.0   
04-280 University Hills                         66.0   151.0   
05-580 Airport                                  70.0    84.0   
05-582 Chena                                   736.0  1191.0   
05-586 Geist                                   309.0   383.0   
05-587 Lakeview                                 54.0    88.0   
05-588 Pike                                     86.0   137.0   
...                                              ...     ...   
District 37 - Absentee                         494.0   689.0   
District 37 - Early Voting                      28.0    48.0   
District 37 - Question                          85.0   171.0   
District 38 - Absentee                         289.0   255.0   
District 38 - Early Voting                      14.0    15.0   
District 38 - Question                         184.0   192.0   
District 39 - Absentee                         202.0   242.0   
District 39 - Early Voting                     102.0   175.0   
District 39 - Question                          91.0   127.0   
District 4 - Absentee                          407.0   726.0   
District 4 - Early Voting                      455.0   798.0   
District 4 - Question                          109.0   276.0   
District 40 - Absentee                         207.0   303.0   
District 40 - Early Voting                      19.0    15.0   
District 40 - Question                          62.0   134.0   
District 5 - Absentee                          334.0   604.0   
District 5 - Early Voting                      329.0   550.0   
District 5 - Question                           92.0   244.0   
District 6 - Absentee                          705.0   754.0   
District 6 - Early Voting                      175.0   269.0   
District 6 - Question                          114.0   207.0   
District 7 - Absentee                          422.0   369.0   
District 7 - Early Voting                      732.0   545.0   
District 7 - Question                          200.0   291.0   
District 8 - Absentee                          410.0   354.0   
District 8 - Early Voting                      396.0   349.0   
District 8 - Question                          117.0   249.0   
District 9 - Absentee           

In [57]:
GENR14votes.join([GENR14totals, GENR14pct, GENR14turnout])

ValueError: Indexes have overlapping values: [('Ballot Measure 2 - 13PSUM', 'NO', 'NP'), ('Ballot Measure 2 - 13PSUM', 'YES', 'NP'), ('Ballot Measure 3 - 13MINW', 'NO', 'NP'), ('Ballot Measure 3 - 13MINW', 'YES', 'NP'), ('Ballot Measure 4 - 12BBAY', 'NO', 'NP'), ('Ballot Measure 4 - 12BBAY', 'YES', 'NP'), ('Dis Court JD1 Miller', 'NO', 'NP'), ('Dis Court JD1 Miller', 'YES', 'NP'), ('Dis Court JD3 Chung', 'NO', 'NP'), ('Dis Court JD3 Chung', 'YES', 'NP'), ('Dis Court JD3 Clark', 'NO', 'NP'), ('Dis Court JD3 Clark', 'YES', 'NP'), ('Dis Court JD3 Estelle', 'NO', 'NP'), ('Dis Court JD3 Estelle', 'YES', 'NP'), ('Dis Court JD3 Illsley', 'NO', 'NP'), ('Dis Court JD3 Illsley', 'YES', 'NP'), ('Dis Court JD3 Motyka', 'NO', 'NP'), ('Dis Court JD3 Motyka', 'YES', 'NP'), ('Dis Court JD3 Rhoades', 'NO', 'NP'), ('Dis Court JD3 Rhoades', 'YES', 'NP'), ('Dis Court JD3 Wolfe', 'NO', 'NP'), ('Dis Court JD3 Wolfe', 'YES', 'NP'), ('Dis Court JD4 Seekins', 'NO', 'NP'), ('Dis Court JD4 Seekins', 'YES', 'NP')

In [67]:
#Test final data_clean function
data_clean(GENR14)

Race                       Ballot Measure 2 - 13PSUM            \
Statistic                                         NO       YES   
Party                                             NP        NP   
Precinct                                                         
01-446 Aurora                               0.452193  0.547807   
01-455 Fairbanks No. 1                      0.380208  0.619792   
01-465 Fairbanks No. 2                      0.339683  0.660317   
01-470 Fairbanks No. 3                      0.424547  0.575453   
01-475 Fairbanks No. 4                      0.387097  0.612903   
01-480 Fairbanks No. 5                      0.491826  0.508174   
01-485 Fairbanks No. 6                      0.386059  0.613941   
01-490 Fairbanks No. 7                      0.383621  0.616379   
01-495 Fairbanks No. 10                     0.473404  0.526596   
02-345 Badger No. 2                         0.409950  0.590050   
02-355 Fairbanks No. 8                      0.451557  0.548443   
02-365 Fairbanks No. 9                      0.489437  0.510563   
02-375 Fort Wainwright                      0.535714  0.464286   
03-130 Badger No. 1                         0.476699  0.523301   
03-135 Chena Lakes                          0.539859  0.460141   
03-165 Newby                                0.516493  0.483507   
03-175 North Pole                           0.546490  0.453510   
03-183 Plack                                0.504193  0.495807   
04-230 Ester                                0.355263  0.644737   
04-240 Farmers Loop                         0.420635  0.579365   
04-250 Goldstream No. 1                     0.289417  0.710583   
04-260 Goldstream No. 2                     0.293463  0.706537   
04-265 Steese East-Gilmore                  0.488424  0.511576   
04-270 Steese West                          0.440578  0.559422   
04-280 University Hills                     0.304147  0.695853   
05-580 Airport                              0.454545  0.545455   
05-582 Chena                                0.381941  0.618059   
05-586 Geist                                0.446532  0.553468   
05-587 Lakeview                             0.380282  0.619718   
05-588 Pike                                 0.385650  0.614350   
...                                              ...       ...   
District 37 - Absentee                      0.417582  0.582418   
District 37 - Early Voting                  0.368421  0.631579   
District 37 - Question                      0.332031  0.667969   
District 38 - Absentee                      0.531250  0.468750   
District 38 - Early Voting                  0.482759  0.517241   
District 38 - Question                      0.489362  0.510638   
District 39 - Absentee                      0.454955  0.545045   
District 39 - Early Voting                  0.368231  0.631769   
District 39 - Question                      0.417431  0.582569   
District 4 - Absentee                       0.359223  0.640777   
District 4 - Early Voting                   0.363128  0.636872   
District 4 - Question                       0.283117  0.716883   
District 40 - Absentee                      0.405882  0.594118   
District 40 - Early Voting                  0.558824  0.441176   
District 40 - Question                      0.316327  0.683673   
District 5 - Absentee                       0.356077  0.643923   
District 5 - Early Voting                   0.374289  0.625711   
District 5 - Question                       0.273810  0.726190   
District 6 - Absentee                       0.483208  0.516792   
District 6 - Early Voting                   0.394144  0.605856   
District 6 - Question                       0.355140  0.644860   
District 7 - Absentee                       0.533502  0.466498   
District 7 - Early Voting                   0.573218  0.426782   
District 7 - Question                       0.407332  0.592668   
District 8 - Absentee                       0.536649  0.463351   
District 8 - Early Voting               

In [72]:
PRIM14

,0,1,2,3,4,5,6,7,8
0,01-446 Aurora,Race Statistics,NP,NP,Number of Precincts,NP,Total,1.0,NaN
1,01-446 Aurora,Race Statistics,NP,NP,Number of Precincts Reporting,NP,Total,1.0,NaN
2,01-446 Aurora,US SENATOR (ADL),ADL,NP,Number of Precincts for Race,NP,Total,1.0,NaN
3,01-446 Aurora,US SENATOR (ADL),ADL,NP,Number of Precincts Reporting,NP,Total,1.0,NaN
4,01-446 Aurora,US SENATOR (ADL),ADL,NP,Registered Voters,NP,Total,2393.0,NaN
5,01-446 Aurora,US SENATOR (ADL),ADL,NP,Times Counted,NP,Total,251.0,NaN
6,01-446 Aurora,US SENATOR (ADL),ADL,NP,"Begich, Mark",DEM,Total,204.0,NaN
7,01-446 Aurora,US SENATOR (ADL),ADL,NP,"Bryk, William \Bill\""""",DEM,Total,2.0,NaN
8,01-446 Aurora,US SENATOR (ADL),ADL,NP,"Fish, Mark S.",LIB,Total,2.0,NaN
9,01-446 Aurora,US SENATOR (ADL),ADL,NP,"Kile, Zachary A.",AI,Total,6.0,NaN


In [73]:
GENR16

,0,1,2,3,4,5,6
0,01-446 Aurora,Race Statistics,Number of Precincts,NP,Total,1,NaN
1,01-446 Aurora,Race Statistics,Number of Precincts Reporting,NP,Total,1,NaN
2,01-446 Aurora,US PRESIDENT,Number of Precincts for Race,NP,Total,1,NaN
3,01-446 Aurora,US PRESIDENT,Number of Precincts Reporting,NP,Total,1,NaN
4,01-446 Aurora,US PRESIDENT,Registered Voters,NP,Total,2486,NaN
5,01-446 Aurora,US PRESIDENT,Times Counted,NP,Total,874,NaN
6,01-446 Aurora,US PRESIDENT,"Castle, Darrell L.",CON,Total,5,NaN
7,01-446 Aurora,US PRESIDENT,"Clinton, Hillary",DEM,Total,295,NaN
8,01-446 Aurora,US PRESIDENT,"De La Fuente, Roque",NA,Total,7,NaN
9,01-446 Aurora,US PRESIDENT,"Johnson, Gary",LIB,Total,75,NaN


In [74]:
PRIM16

,0,1,2,3,4,5,6,7,8
0,01-446 Aurora,Race Statistics,NP,NP,Number of Precincts,NP,Total,1,NaN
1,01-446 Aurora,Race Statistics,NP,NP,Number of Precincts Reporting,NP,Total,1,NaN
2,01-446 Aurora,US SENATOR (ADL),ADL,NP,Number of Precincts for Race,NP,Total,1,NaN
3,01-446 Aurora,US SENATOR (ADL),ADL,NP,Number of Precincts Reporting,NP,Total,1,NaN
4,01-446 Aurora,US SENATOR (ADL),ADL,NP,Registered Voters,NP,Total,2428,NaN
5,01-446 Aurora,US SENATOR (ADL),ADL,NP,Times Counted,NP,Total,61,NaN
6,01-446 Aurora,US SENATOR (ADL),ADL,NP,"Blatchford, Edgar",DEM,Total,25,NaN
7,01-446 Aurora,US SENATOR (ADL),ADL,NP,"Metcalfe, Ray",DEM,Total,25,NaN
8,01-446 Aurora,US SENATOR (ADL),ADL,NP,"Stevens, Cean",LIB,Total,7,NaN
9,01-446 Aurora,US SENATOR (R),REP,NP,Number of Precincts for Race,NP,Total,1,NaN


In [4]:
GENR14raw.to_string()

KeyboardInterrupt: 

In [8]:
GENR14raw.name

AttributeError: 'DataFrame' object has no attribute 'name'

In [11]:
print GENR14raw.index.name

None


In [20]:
GENR14raw.columns

Int64Index([0, 1, 2, 3, 4, 5, 6], dtype='int64')